In [1]:
import urllib2
import json
import re
import io
import copy
import nltk, re, pprint
import os
import time
import calendar
import datetime
from threading import Thread
from time import sleep
from __future__ import division
import codecs

<h1> Select the right movies from all movies

In [ ]:
i = 0
for filename in os.listdir('Testing'):
    test = io.open("Testing/" + filename, 'r', encoding='utf-8')
    sidensJson = json.load(test)
    #print sidensJson["release_date"]
    try:
        if (sidensJson["budget"] > 10000 and sidensJson["revenue"] > 10000 and int(sidensJson["release_date"][:4]) > 1976 and sidensJson["vote_count"] > 0 and sidensJson["runtime"] > 0 and sidensJson["original_language"] == "en"):
            i = i + 1
            #with open('DoneMovies/' + filename, 'w') as outfile:
                #json.dump(sidensJson, outfile)
    except:
        print filename
print i

<h1> Get Data from MovieDB

In [ ]:
f = io.open('data.tsv', 'r', encoding='utf-8')
movies = re.findall(r'\w\w\w\w\w\w\w\w\w\tmovie.+\n', f.read())
f.close()
f = io.open('data.tsv', 'r', encoding='utf-8')
documentarys = re.findall(r'\w\w\w\w\w\w\w\w\w\tmovie.+\tDocumentary\n', f.read())
f.close()
f = io.open('data.tsv', 'r', encoding='utf-8')
empty = re.findall(r'\w\w\w\w\w\w\w\w\w\tmovie.+\t\\N\n', f.read())
f.close()
movieSet = set(movies)
docuSet = set(documentarys)
emptyMovieSet = set(empty)
KunMovies = (movieSet - docuSet) - emptyMovieSet

In [ ]:
f = io.open('Movies.txt', 'w', encoding='utf-8')
for i in KunMovies:
    f.write(i)
f.close()
f = io.open('Movies.txt', 'r', encoding='utf-8')
IMDB = re.findall(r'(\w\w\w\w\w\w\w\w\w).+\n', f.read())

In [ ]:
def threaded_function(arg):
    f = io.open('Done2.txt', 'a', encoding='utf-8')    
    f.writelines(arg + "\n")
    f.close()
    try:
        query = "https://api.themoviedb.org/3/movie/" + arg + "?api_key=165c6cb5ab10a81562b9e48c4dff4665&language=en-US"
        wikiresponse = urllib2.urlopen(query)
        sidensJson = json.load(wikiresponse)
        revenue = sidensJson["revenue"]
        if (revenue > 0):
            print "NUMMERET ER " , IMDB.index(arg)
            with open('Testing/' + arg + '.txt', 'w') as outfile:
                json.dump(sidensJson, outfile)
            f = io.open('Success.txt', 'a', encoding='utf-8')    
            f.writelines(arg + "\n")
            f.close()
    except:
        pass

In [ ]:
thread = Thread(target = threaded_function, args = (IMDB[0]))
Threads = []
startTime = calendar.timegm(time.gmtime())
doneList = []
f = io.open('Done.txt', 'r')
doneList = f.readlines()
doneList = [x.strip() for x in doneList] 
f.close()
q = 0
for i in range(0,len(IMDB)):
    try:
        doneList.index(IMDB[i])
    except:
        if (q % 40 == 0 and q != 0):
            for j in range(q-40, q):
                Threads[j].join()
        thread = Thread(target = threaded_function, args = (IMDB[i], ))
        print i
        Threads.append(thread)
        Threads[q].start()

        if (q % 40 == 0 and q != 0):
            for j in Threads:
                j.join()
            slutTime = calendar.timegm(time.gmtime())
            diffTime = slutTime - startTime
            #print "Difftime = " , diffTime
            if (diffTime <= 10):
                strafTid = 12 - diffTime
                #print "Pause = ", strafTid
                time.sleep(strafTid)
            startTime = calendar.timegm(time.gmtime())
        q = q + 1

<h1> Reviews <h1>
<h3> We try to get the 10 best rated reviews by users on IMDB. If there are less than 10 for the individual movie, we take what we can get.

In [ ]:
import urllib2
import codecs
from HTMLParser import HTMLParser
import os
totalNumberOfFiles = len(os.listdir("DoneMovies/"))
countt = 0
for f in os.listdir("DoneMovies/"):
    if f[-4:] == ".txt" and (not os.path.isfile("DoneMoviesReviews/" + f) or os.path.getsize("DoneMoviesReviews/" + f) == 0):
        idd = f[:-4]

        response = urllib2.urlopen('http://www.imdb.com/title/' + idd + '/reviews')
        html2 = response.read()

        div = html2.split("tn15content")[1]
        parts = div.split("<b>Author:</b>")
        parts.pop(0)
        h = HTMLParser()
        res = []
        for part in parts:
            try:
                dataa = h.unescape(part.split("</div>\n<p>")[1].split("</p>")[0].replace("<br>", "\n"))
                f = codecs.open("temp.txxt", "w", "utf-8")
                f.write("<|_=00=_|>".join([dataa]))
                f.close()
                res.append(dataa)
            except Exception as e:
                pass            
        try:
            f = codecs.open("DoneMoviesReviews/" + idd + ".txt", "w", "utf-8")
            f.write("<|_=00=_|>".join(res))
            f.close()
        except Exception as e:
            pass
    countt += 1
    print countt / float(totalNumberOfFiles)

In [ ]:
y = 0
for f in os.listdir("DoneMovies/"):
    if f[-4:] == ".txt" and (not os.path.isfile("DoneMoviesReviews/" + f)):                          
        try:
            os.remove("DoneMovies/" + f)
            y = y+1
        except OSError:
            pass
print y   

<h1> Get Credits

In [ ]:
def threaded_functionForCredits(arg):
    try:
        query = "https://api.themoviedb.org/3/movie/" + str(arg) + "/credits?api_key=165c6cb5ab10a81562b9e48c4dff4665&language=en-US"
        wikiresponse = urllib2.urlopen(query)
        sidensJson = json.load(wikiresponse)
        with open('Credits/' + str(arg) + '.txt', 'w') as outfile:
            json.dump(sidensJson, outfile)
    except:
        print arg, "Fejl"
        pass

In [ ]:
q = 0
Threads = []
startTime = calendar.timegm(time.gmtime())
for filename in os.listdir('DoneMovies'):
    test = io.open("DoneMovies/" + filename, 'r', encoding='utf-8')
    sidensJson = json.load(test)
    if (q % 40 == 0 and q != 0):
        for j in range(q-40, q):
            Threads[j].join()
    thread = Thread(target = threaded_functionForCredits, args = (sidensJson["id"], ))
    Threads.append(thread)
    Threads[q].start()
    if (q % 40 == 0 and q != 0):
        for j in Threads:
            j.join()
        slutTime = calendar.timegm(time.gmtime())
        diffTime = slutTime - startTime
        #print "Difftime = " , diffTime
        if (diffTime <= 10):
            strafTid = 12 - diffTime
            #print "Pause = ", strafTid
            time.sleep(strafTid)
        startTime = calendar.timegm(time.gmtime())
    print q
    q = q + 1

<h1> Get all Actors from Credits

In [ ]:
for filename in os.listdir('Credits'):
    test = io.open("Credits/" + filename, 'r', encoding='utf-8')
    sidensJson = json.load(test)
    test.close()
    castList = []
    for i in sidensJson["cast"]:
        if(i["character"].rfind("uncredited") == -1):
            try:
                if str(i["id"]) not in castList:
                    f = io.open("Actors/" + str(i["id"]) + ".txt", 'a', encoding='utf-8')
                    f.write(unicode(filename[:-4]) + "\n")
                    f.close()
                    castList.append(str(i["id"]))
                else:
                    pass
            except:
                f = io.open("Actors/" + str(i["id"]) + ".txt", 'w', encoding='utf-8')
                f.write(unicode(filename[:-4]) + "\n")
                f.close()
                castList.append(str(i["id"]))
    #break

<h1> Get all directors from Credits

In [ ]:
for filename in os.listdir('Credits'):
    test = io.open("Credits/" + filename, 'r', encoding='utf-8')
    sidensJson = json.load(test)
    test.close()
    castList = []
    for i in sidensJson["crew"]:
        if i["job"] == "Director":
            try:
                if str(i["id"]) not in castList:
                    f = io.open("Directors/" + str(i["id"]) + ".txt", 'a', encoding='utf-8')
                    f.write(unicode(filename[:-4]) + "\n")
                    f.close()
                    castList.append(str(i["id"]))
                else:
                    pass
            except:
                f = io.open("Directors/" + str(i["id"]) + ".txt", 'w', encoding='utf-8')
                f.write(unicode(filename[:-4]) + "\n")
                f.close()
                castList.append(str(i["id"]))
    #break

<h1> Actors in more than 1 movie

In [ ]:
i = 0
for filename in os.listdir('Actors'):
    test = io.open("Actors/" + filename, 'r', encoding='utf-8')
    indhold = test.readlines()
    if(len(indhold) > 1):
        i = i + 1
        f = io.open("ActorsMoreThan1/" + filename, 'w', encoding='utf-8')
        f.writelines(indhold)
        f.close()
    test.close()
print i

<h1> Actors in more than X movies

In [ ]:
i = 0
for filename in os.listdir('Actors'):
    test = io.open("Actors/" + filename, 'r', encoding='utf-8')
    indhold = test.readlines()
    if(len(indhold) >= 10):
        i = i + 1
        f = io.open("ActorsMoreThanX/" + filename, 'w', encoding='utf-8')
        f.writelines(indhold)
        f.close()
    test.close()
print i

<h1> Get Json for all Actors/Directors

In [ ]:
def threaded_functionForPersonJson(arg):
    try:
        query = "https://api.themoviedb.org/3/person/" + str(arg) + "?api_key=165c6cb5ab10a81562b9e48c4dff4665&language=en-US"
        wikiresponse = urllib2.urlopen(query)
        sidensJson = json.load(wikiresponse)
        with open('PersonJson/' + str(arg) + '.txt', 'w') as outfile:
            json.dump(sidensJson, outfile)
    except:
        print arg, "Fejl"
        pass

In [ ]:
q = 0
Threads = []
startTime = calendar.timegm(time.gmtime())
for filename in os.listdir('Directors'):
    if (q % 40 == 0 and q != 0):
        for j in range(q-40, q):
            Threads[j].join()
    thread = Thread(target = threaded_functionForPersonJson, args = (filename[:-4], ))
    Threads.append(thread)
    Threads[q].start()
    if (q % 40 == 0 and q != 0):
        for j in Threads:
            j.join()
        slutTime = calendar.timegm(time.gmtime())
        diffTime = slutTime - startTime
        #print "Difftime = " , diffTime
        if (diffTime <= 10):
            strafTid = 12 - diffTime
            #print "Pause = ", strafTid
            time.sleep(strafTid)
        startTime = calendar.timegm(time.gmtime())
    print q
    q = q + 1
for filename in os.listdir('ActorsInMoreThanX'):
    if (q % 40 == 0 and q != 0):
        for j in range(q-40, q):
            Threads[j].join()
    thread = Thread(target = threaded_functionForPersonJson, args = (filename[:-4], ))
    Threads.append(thread)
    Threads[q].start()
    if (q % 40 == 0 and q != 0):
        for j in Threads:
            j.join()
        slutTime = calendar.timegm(time.gmtime())
        diffTime = slutTime - startTime
        #print "Difftime = " , diffTime
        if (diffTime <= 10):
            strafTid = 12 - diffTime
            #print "Pause = ", strafTid
            time.sleep(strafTid)
        startTime = calendar.timegm(time.gmtime())
    print q
    q = q + 1

<h1> Actor pairs

In [ ]:
count = 0
nr1 = 0
nr2 = 0
countnr = 0
for filename in os.listdir('ActorsMoreThanX'):
    nr2 = 0
    #print filename[:-4]
    test = io.open("ActorsMoreThanX/" + filename, 'r', encoding='utf-8')
    doneList = test.readlines()
    doneList = [x.strip() for x in doneList]
    test.close()
    for filename2 in os.listdir('ActorsMoreThanX'):
        if (nr2 > nr1):
            if str(filename[:-4]) != str(filename2[:-4]):
                #print filename, filename2
                matches = []
                test2 = io.open("ActorsMoreThanX/" + filename2, 'r', encoding='utf-8')
                doneList2 = test2.readlines()
                doneList2 = [x.strip() for x in doneList2]
                test2.close()
                for i in doneList:
                    for j in doneList2:
                        if i == j:
                            #print i == j
                            matches.append(str(i))
                if(len(matches) >= 3):
                    countnr = countnr + 1
                    f = io.open("ActorPairs/" + filename[:-4] + "-" + filename2, 'w', encoding='utf-8')
                    for i in matches:
                        f.writelines(unicode(i) + "\n")
                    f.close()
        nr2 = nr2 + 1
        #break
    print count
    count = count + 1
    nr1 = nr1 + 1
print countnr

<h1> Director and Actor Pair

In [ ]:
count = 0
countnr = 0
for filename in os.listdir('ActorsMoreThanX'):
    #print filename[:-4]
    test = io.open("ActorsMoreThanX/" + filename, 'r', encoding='utf-8')
    doneList = test.readlines()
    doneList = [x.strip() for x in doneList]
    test.close()
    for filename2 in os.listdir('Directors'):
            if str(filename[:-4]) != str(filename2[:-4]):
                #print filename, filename2
                matches = []
                test2 = io.open("Directors/" + filename2, 'r', encoding='utf-8')
                doneList2 = test2.readlines()
                doneList2 = [x.strip() for x in doneList2]
                test2.close()
                for i in doneList:
                    for j in doneList2:
                        if i == j:
                            #print i == j
                            matches.append(str(i))
                if(len(matches) >= 2):
                    countnr = countnr + 1
                    f = io.open("ActorDirectorPair/" + filename[:-4] + "-" + filename2, 'w', encoding='utf-8')
                    for i in matches:
                        f.writelines(unicode(i) + "\n")
                    f.close()
        #break
    print count
    count = count + 1
print countnr

<h1> Get all genres for each movie.

In [ ]:
movieAproved = []
for f in os.listdir("DoneMovies/"):
    movieAproved.append(f[:-4])

In [ ]:
movieGenreList = []
with open('Movies.txt','r') as f:
    for line in f:
        if line.split(None, 1)[0] in movieAproved:
            moviegenre = re.search(r'(\w\w\w\w\w\w\w\w\w)\t.+\t([\w,-]+)\n', line)
            if moviegenre:
                #print "moviegenre.group() : ",  moviegenre.group(1) +" "+ moviegenre.group(2)
                movieGenreList.append(moviegenre.group(1) +" "+ moviegenre.group(2))
            else:
                print ("None Type:"+line.split(None, 1)[0])

In [ ]:
for moviegenres in movieGenreList:
    splitted = moviegenres.split()
    movieid = splitted[0]
    moviegen = splitted[1]
    if (not os.path.isfile("MovieGenres/" + movieid +".txt") or os.path.getsize("MovieGenres/" + movieid +".txt") == 0):
        f = codecs.open("MovieGenres/" + movieid +".txt", "w", "utf-8")
        f.write(moviegen)
        f.close()

In [ ]:
for filename in os.listdir('MovieGenres'):
    with open("MovieGenres/" + filename) as movie:
        genres = movie.read().split(',')
        for gen in genres:
            try:
                f = io.open("GenreList/" + gen + ".txt", 'a', encoding='utf-8')
                f.write(unicode(filename[:-4]) + "\n")
                f.close()
            except:
                f = io.open("GenreList/" + gen + ".txt", 'w', encoding='utf-8')
                f.write(unicode(filename[:-4]) + "\n")
                f.close()
        